In [1]:
# import sys
# sys.path.append('/Users/ambershen/Desktop/linARG/software/stdpopsim/stdpopsim')
import stdpopsim
import pandas as pd
import scipy.sparse as sp
import numpy as np
import tskit

In [2]:
# region in the format of chrXX-XXX-XXX
def simulate_OutOfAfrica_3G09(sample_size, region, seed, out):
    n = int(sample_size/3)
    chrom, start, end = region.split('-')
    species = stdpopsim.get_species('HomSap')
    model = species.get_demographic_model('OutOfAfrica_3G09')
    contig = species.get_contig(chrom, mutation_rate=model.mutation_rate, inclusion_mask=[(start, end)])
    samples = {'YRI': n, 'CHB': n, 'CEU': n}
    engine = stdpopsim.get_engine('msprime')    
    ts = engine.simulate(model, contig, samples, seed=seed)
    ts.dump(out)

In [3]:
simulate_OutOfAfrica_3G09(1e3, 'chr21-21990355-23324211', 1, 'test.trees')
ts = tskit.load('test.trees')
sum([1 for x in ts.mutations()]), sum([1 for x in ts.variants()])

(19477, 19477)

Solution: add discrete_genome=False to mutation simulator

In [172]:
def get_genotype_matrix(ts):
    X_sites = sp.csr_matrix(ts.genotype_matrix()) # sites x samples
    X_mut = []
    counter = 0
    for site in X_sites:
        alleles = set(site.data)
        if len(alleles) == 1:
            X_mut.append(site)
            counter += 1
        else:
            counter += len(alleles)
            for allele in alleles:
                allele_inds = site.indices[np.where(site.data==allele)[0]] # which individuals have the allele
                data = np.ones(len(allele_inds))
                row_inds = np.zeros(len(allele_inds), dtype=int)
                row = sp.csr_matrix((data, (row_inds, allele_inds)), shape=(1, X_sites.shape[1]))
                X_mut.append(row)
    print(counter)
    return sp.vstack(X_mut).T
                    

In [173]:
X = get_genotype_matrix(ts)

19507


In [156]:
X.shape

(1998, 19507)

In [157]:
sum([1 for x in ts.mutations()]), sum([1 for x in ts.variants()])

(19565, 19404)

In [181]:
# simulate_OutOfAfrica_3G09(1e3, 'chr21-21990355-23324211', 1, 'test.trees')
ts = tskit.load('test.trees')
sum([1 for x in ts.mutations()]), sum([1 for x in ts.variants()])

(19477, 19316)

In [93]:
region = 'chr21-21990355-23324211'
seed = 1
n = int(1e3/3)
chrom, start, end = region.split('-')
species = stdpopsim.get_species('HomSap')
model = species.get_demographic_model('OutOfAfrica_3G09')
contig = species.get_contig(chrom, mutation_rate=model.mutation_rate, inclusion_mask=[(start, end)])
samples = {'YRI': n, 'CHB': n, 'CEU': n}
engine = stdpopsim.get_engine('msprime')    
ts = engine.simulate(model, contig, samples, seed=seed, discrete_genome=False)

In [94]:
contig

Contig(recombination_map=RateMap(position=array([       0., 46709983.]), rate=array([1.72443408e-08])), mutation_rate=2.35e-08, ploidy=2, bacterial_recombination=False, gene_conversion_fraction=None, gene_conversion_length=None, genetic_map=None, inclusion_mask=array([[21990355, 23324211]]), exclusion_mask=None, dfe_list=[DFE(id='neutral', description='neutral DFE', long_description='strictly neutral mutations', mutation_types=[MutationType(dominance_coeff=0.5, distribution_type='f', distribution_args=[0], convert_to_substitution=True)], proportions=[1.0], citations=[], qc_dfe=None)], interval_list=[array([[       0, 46709983]])], original_coordinates=('chr21', 0, 46709983))

In [91]:
import tskit

ts = tskit.load('test.trees')

In [92]:
sum([1 for x in ts.mutations()]), sum([1 for x in ts.variants()])

(19565, 19404)

In [176]:
for x in ts.variants():
    if x.site.id == 91:
        print(x)
        break

╔════════════════════════════════════╗
║Variant                             ║
╠═══════════════════════╤════════════╣
║Site id                │          91║
╟───────────────────────┼────────────╢
║Site position          │21,997,610.0║
╟───────────────────────┼────────────╢
║Number of samples      │       1,998║
╟───────────────────────┼────────────╢
║Number of alleles      │           3║
╟───────────────────────┼────────────╢
║Samples with allele 'C'│   724 (36%)║
╟───────────────────────┼────────────╢
║Samples with allele 'A'│ 1,270 (64%)║
╟───────────────────────┼────────────╢
║Samples with allele 'T'│    4 (0.2%)║
╟───────────────────────┼────────────╢
║Has missing data       │       False║
╟───────────────────────┼────────────╢
║Isolated as missing    │        True║
╚═══════════════════════╧════════════╝



In [ ]:
site_to_mutation = {i:[] for i in range()}
for x in ts.sites():
    
    if len(x.mutations)>1:
        break
x.mutations

[Mutation(id=90, site=90, node=466, derived_state='A', parent=-1, metadata=b'', time=258.3545654903446, edge=23699),
 Mutation(id=91, site=90, node=2656, derived_state='A', parent=-1, metadata=b'', time=129.49810280336771, edge=22907)]

In [159]:
def get_tree_sequence_adj_mtx(ts):
    mutations = ts.mutations()
    sites = ts.sites()
    site_positions = {site.id: site.position for site in sites}
    mutations_data = []
    for mutation in mutations:
        mutations_data.append({
            'mutation_id': mutation.id,
            'site_position': site_positions[mutation.site],
            'node': mutation.node,
            'time': mutation.time
        })
    mutations_df = pd.DataFrame(mutations_data)

    edges = pd.DataFrame({
        'left': ts.tables.edges.left,
        'right': ts.tables.edges.right,
        'parent': ts.tables.edges.parent,
        'child': ts.tables.edges.child,
    })
            
    tree_sequence = {}
    nodes = set(list(edges.parent) + list(edges.child))
    N = len(nodes)
    assert N == max(nodes)+1
    intervals = sorted(list(set(list(edges.left) + list(edges.right))))
    for i in range(len(intervals)-1):
        left_interval = intervals[i]
        right_interval = intervals[i+1]
        edges_in_interval = edges[(edges.left<=left_interval) & (edges.right>=right_interval)]
        adj_mat = sp.csr_matrix((np.ones(edges_in_interval.shape[0]), (np.array(edges_in_interval.parent), np.array(edges_in_interval.child))), shape=(N, N))
        tree_sequence[(left_interval,right_interval)] = adj_mat
    
    return tree_sequence, mutations_df

In [160]:
tree_sequence, mutations = get_tree_sequence_adj_mtx(ts)

In [166]:
np.sum(list(mutations.site_position.value_counts()))

19565

In [170]:
for x in ts.mutations():
    print(x)
    break

Mutation(id=0, site=0, node=1865, derived_state='A', parent=-1, metadata=b'', time=15.120195115225497, edge=5385)


In [171]:
mutations

,mutation_id,site_position,node,time
0,0,21990418.0,1865,15.120195
1,1,21990472.0,12268,7581.733773
2,2,21990571.0,11703,10919.025760
3,3,21990699.0,11012,10409.829788
4,4,21990723.0,9960,3604.132516
...,...,...,...,...
19560,19560,23324027.0,238,659.127112
19561,19561,23324108.0,153,7.281762
19562,19562,23324129.0,95,126.591971
19563,19563,23324141.0,996,551.702964
